In [2]:
# Created on Tue Feb  7 13:59:25 2023
# Author: earyo

# In this script we fit a Pareto-lognormal distribution to the wealth distribution
# data in the USA from https://realtimeinequality.org/

import os
from inequality_metrics import find_wealth_groups2
import numpy as np
from scipy.stats import powerlognorm, genpareto
import matplotlib.pyplot as plt
import seaborn as sns 
from tqdm import tqdm

# Empirical wealth shares in the UNITED STATES in January 2019
empirical_wealth_shares = [34.8, 70.9, 28.8, 0.2]
average_US_wealth_per_adult = 410400

# Empirical wealth shares for January 1990
empirical_wealth_shares_1990 = [28.6, 64.7, 33.4, 1.8]
average_US_wealth_per_adult_1990 = 203900

def PLN_normalized(c, s, sample_size):   
    mean, var, skew, kurt = powerlognorm.stats(c, s, moments='mvsk')
    r = powerlognorm.rvs(c, s, size=sample_size)
    w = find_wealth_groups2(r, sum(r))[1]
    w_percent = [x*100 for x in w]
    return w_percent, r, mean

def plot_wealth_groups(bars1, bars2): 
    labels = ['top1%', 'top10%', 'next40%', 'bottom50%']
    x = np.arange(len(labels))
    width = 0.35
    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, bars1, width, label='empirical')
    rects2 = ax.bar(x + width/2, bars2, width, label='model')
    ax.set_ylabel('% of wealth in the USA')
    ax.set_xticks(x, labels)
    ax.legend()
    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)
    fig.tight_layout()
    return fig
    
def optimal_fit_PLN(c_range, s_range, sample_size, empirical_distr):
    q = empirical_distr
    result_range = np.zeros((len(c_range), len(s_range)))
    for i in tqdm(range(len(c_range))):
        for j in range(len(s_range)):
            r = powerlognorm.rvs(c_range[i], s_range[j], size=sample_size)
            w = find_wealth_groups2(r, sum(r))[1]
            z = [x*100 for x in w]
            minimization_fct = sum([abs(z[x] - q[x]) for x in range(4)])
            result_range[i, j] = minimization_fct    
    return result_range

def optimal_fit_GP(eta_range, sample_size, empirical_distr):
    q = empirical_distr
    result_range = np.zeros(len(eta_range))
    for i in tqdm(range(len(eta_range))):
        r = genpareto.rvs(eta_range[i], size=sample_size)
        w = find_wealth_groups2(r, sum(r))[1]
        z = [x*100 for x in w]
        minimization_fct = sum([abs(z[x] - q[x]) for x in range(4)])
        result_range[i] = minimization_fct    
    return result_range

def heatmap2d(arr, xticks, yticks):
    plt.imshow(arr, cmap='viridis')
    plt.title('Minimize abs error of percentages between wealth groups')
    plt.colorbar()
    plt.xlabel("parameter s")
    plt.ylabel("parameter c")
    plt.xticks(np.linspace(0, 99, 10), yticks, rotation="vertical")
    plt.yticks(np.linspace(0, 99, 10), xticks)        
    plt.show()

def plot_results_GP(eta, sample_size, empirical_wealth_shares):
    sampled_distr = genpareto.rvs(eta, size=sample_size)
    groups_modelled = find_wealth_groups2(sampled_distr, sum(sampled_distr))[1]
    groups_modelled_percent = [x * 100 for x in groups_modelled]
    plot = plot_wealth_groups(empirical_wealth_shares, groups_modelled_percent)
    plot.suptitle(f"GP-model opt fit for eta={eta} vs. empirical data", y=1.05)
    plt.show()

def main():
    c_range_data = np.around(np.linspace(0.1, 2, 100), 2)
    s_range_data = np.around(np.linspace(0.5, 2.5, 100), 2)
    sample_size = 10**4

    results = optimal_fit_PLN(c_range_data, s_range_data, sample_size, empirical_wealth_shares)
    optimal_c = c_range_data[np.where(results == np.min(results))[0][0]]   
    optimal_s = s_range_data[np.where(results == np.min(results))[1][0]]
    c_range_ticks = list(c_range_data)[::10]
    s_range_ticks = list(s_range_data)[::10]
    heatmap2d(results, c_range_ticks, s_range_ticks)

    results_1990 = optimal_fit_PLN(c_range_data, s_range_data, sample_size, empirical_wealth_shares_1990)
    optimal_c_1990 = c_range_data[np.where(results_1990 == np.min(results_1990))[0][0]]   
    optimal_s_1990 = s_range_data[np.where(results_1990 == np.min(results_1990))[1][0]]
    heatmap2d(results_1990, c_range_ticks, s_range_ticks)

    sampled_distr = PLN_normalized(0.33, 1.15, sample_size)
    groups_modelled, raw_sample, mean = sampled_distr
    plot1 = plot_wealth_groups(empirical_wealth_shares, groups_modelled)
    plot1.suptitle("PLN-model opt fit 2019 vs. empirical data", y=1.05)
    plt.show()

    sampled_distr2 = PLN_normalized(1.92, 2.08, sample_size)
    groups_modelled2, raw_sample2, mean2 = sampled_distr2
    plot2 = plot_wealth_groups(empirical_wealth_shares_1990, groups_modelled2)
    plot2.suptitle("PLN-model opt fit 1990 vs. empirical data", y=1.05)
    plt.show()

    eta_range = np.around(np.linspace(0.1, 2, 100), 2)
    results_gp = optimal_fit_GP(eta_range, sample_size, empirical_wealth_shares)
    optimal_eta = eta_range[np.argmin(results_gp)]
    print(f'Optimal eta for Generalized Pareto Distribution: {optimal_eta}')

    plot_results_GP(optimal_eta, sample_size, empirical_wealth_shares)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'inequality_metrics'